In [ ]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pylab as plt

<font size=4>This spaceship Titanic prject is imagination provoking. I enjoyed a lot as a SciFi Fan. Hope you like my work. Also have a look at my [notebook](https://www.kaggle.com/satoshiss/titanic-binary-classification) of Titanic, if you have time. Thanks!!</font>

# Read Data

In [ ]:
train_df = pd.read_csv('../input/spaceship-titanic/train.csv')
test_df = pd.read_csv('../input/spaceship-titanic/test.csv')


# Preparing Data

In [ ]:
#make full data
train_df['isTrain'] = True
test_df['isTrain'] = False
full = pd.concat([train_df,test_df])

In [ ]:
#Separate PassengerID into group id and give number of group.

full[['GroupId','member']] = full.PassengerId.str.split('_',expand=True)
full['group_num'] = full.GroupId.map(full.groupby(['GroupId'])['PassengerId'].count().to_dict())

full['group_youngest'] = full.GroupId.map(full.groupby(['GroupId'])['Age'].min().to_dict())
full['group_oldest'] = full.GroupId.map(full.groupby(['GroupId'])['Age'].max().to_dict())
full['group_age_mean'] = full.GroupId.map(full.groupby(['GroupId'])['Age'].mean().to_dict())
full['group_age_std'] = full.GroupId.map(full.groupby(['GroupId'])['Age'].std().to_dict())

In [ ]:
full.query('group_num>1')

In [ ]:
#adding number of destinations within a group

dict =  full.groupby('GroupId').Destination.nunique().to_dict()
full['num_destination'] = full.GroupId.map(dict)

In [ ]:
#separate fulldata back to train_df and test_df
train_df = full.query('isTrain==True')
test_df = full.query('isTrain==False')

In [ ]:
# nan is located many columns so count it for each row.

train_df['nan_count'] = train_df.isna().sum(axis=1)
test_df['nan_count'] = test_df.isna().sum(axis=1)

In [ ]:
train_df.query('nan_count==3')

In [ ]:
#HomePlanet

#print(train_df.HomePlanet.unique(),test_df.HomePlanet.unique())
#['Europa' 'Earth' 'Mars' nan] ['Earth' 'Europa' 'Mars' nan]

train_df.HomePlanet = train_df.HomePlanet.fillna('NoRecord')
test_df.HomePlanet = test_df.HomePlanet.fillna('NoRecord')

home = {'Europa':0,'Earth':1,'Mars':2,'NoRecord':3}


train_df['home_embed'] = train_df.HomePlanet.map(home)
test_df['home_embed'] = test_df.HomePlanet.map(home)


In [ ]:
#CryoSleep
#print(train_df.CryoSleep.unique(),test_df.CryoSleep.unique())
#[False True nan] [True False nan]

train_df.CryoSleep = train_df.CryoSleep.fillna('NoRecord')
test_df.CryoSleep = test_df.CryoSleep.fillna('NoRecord')

sleep = {True:1,False:0,'NoRecord':2}

train_df['sleep_embed'] = train_df.CryoSleep.map(sleep)
test_df['sleep_embed'] = test_df.CryoSleep.map(sleep)

In [ ]:
#separate Cabin into deck,num, and side
train_df['Cabin'] = train_df['Cabin'].fillna('H/2000/N')
test_df['Cabin'] = test_df['Cabin'].fillna('H/2000/N')

train_df[['deck','num','side']]=train_df['Cabin'].str.split('/',expand=True)
test_df[['deck','num','side']] = test_df['Cabin'].str.split('/',expand=True)

#deck
#print(train_df.deck.unique(),test_df.deck.unique())
#['B' 'F' 'A' 'G' nan 'E' 'D' 'C' 'T'] ['G' 'F' 'C' 'B' 'D' 'E' nan 'A' 'T']
# Nan was replaced with H

deck = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7,'H':8}

train_df['deck_embed'] = train_df.deck.map(deck)
test_df['deck_embed'] = test_df.deck.map(deck)

#num
#print(train_df.num.unique(),test_df.num.unique())
#['0' '1' '2' ... '1892' '1893' '1894'] ['3' '4' '0' ... '1890' '1794' '1503']
# Nan was replaced with 2000 for now

train_df['num'] = train_df['num'].astype(int)
test_df['num'] = test_df['num'].astype(int)

#side
#print(train_df.side.unique(),test_df.side.unique())
#['P' 'S' ''] ['S' 'P' nan]
#Nan was replaced with N

side ={'P':0,'S':1,'N':2}

train_df['side_embed'] = train_df.side.map(side)
test_df['side_embed'] = test_df.side.map(side)

In [ ]:
#destination

#print(train_df.Destination.unique(),test_df.Destination.unique())
#['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan] ['TRAPPIST-1e' '55 Cancri e' 'PSO J318.5-22' nan]

train_df.Destination = train_df.Destination.fillna('NoRecord')
test_df.Destination  = test_df.Destination.fillna('NoRecord')

destination = {'TRAPPIST-1e':0, 'PSO J318.5-22':1, '55 Cancri e':2,'NoRecord':3}

train_df['destination_emmbed'] = train_df.Destination.map(destination)
test_df['destination_emmbed'] = test_df.Destination.map(destination)

In [ ]:
#Age 

#train_df.Age.isna().sum(),test_df.Age.isna().sum()
#(179, 91)

age_mean = (train_df.Age.sum() + test_df.Age.sum())/ (len(train_df) +len(test_df)-270)
train_df['isAge'] = -train_df.Age.isna()
test_df['isAge'] = -test_df.Age.isna()

train_df['Age'] = train_df.Age.fillna(age_mean)
test_df['Age'] = test_df.Age.fillna(age_mean)


In [ ]:
#Vip 

#train_df.VIP.unique(), test_df.VIP.unique()
#[True,False,nan] [True,False,nan]
train_df.VIP = train_df.VIP.fillna('NoRecord')
test_df.VIP = test_df.VIP.fillna('NoRecord')

vip = {True:1,False:0,'NoRecord':2}

train_df['vip_embed'] = train_df.CryoSleep.map(vip)
test_df['vip_embed'] = test_df.CryoSleep.map(vip)



In [ ]:
#train average spending
t = train_df.query('CryoSleep==False')
t.RoomService.mean(),t.FoodCourt.mean(),t.ShoppingMall.mean(),t.Spa.mean(),t.VRDeck.mean()

In [ ]:
# Luxury spenditure sum before filling na

train_df['luxury'] = train_df[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].sum(axis=1)
test_df['luxury'] = test_df[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].sum(axis=1)

train_df['luxurywithoutfood'] = train_df[['RoomService','ShoppingMall','Spa','VRDeck']].sum(axis=1)
test_df['luxurywithoutfood'] = test_df[['RoomService','ShoppingMall','Spa','VRDeck']].sum(axis=1)

* <font size =4>  If a passenger is  elected to sleep, he or she does not use luxury stuff.  </font>

In [ ]:
def fill_zero(column):
    train_df[column] = [ 0 if a==True else b for a,b in zip(train_df.CryoSleep,train_df[column])]
    test_df[column] = [ 0 if a==True else b for a,b in zip(test_df.CryoSleep,test_df[column])]
    
    return train_df,test_df

In [ ]:
for i in ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']:
    train_df,test_df = fill_zero(i)

In [ ]:
train_df

In [ ]:
def fill_na(column):
    tr = train_df.query('CryoSleep==False')
    ts = test_df.query('CryoSleep==False')
    mean = (tr[column].sum() + ts[column].sum())/ (len(tr) +len(ts)-(tr[column].isna().sum()+ ts[column].isna().sum()))
    
    train_df['is' + str(column)] = -train_df[column].isna()
    test_df['is' +str(column)] = -test_df[column].isna()

    train_df[column] = train_df[column].fillna(mean)
    test_df[column] = test_df[column].fillna(mean)
    
    return train_df,test_df


In [ ]:
for i in ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']:
    train_df,test_df = fill_na(i)

# Family or Not
<font size=4> Let's find out if a group member shares the last name. </font>

In [ ]:
#Name 
train_df.Name.str.split(' ').str.len().max(), test_df.Name.str.split(' ').str.len().max()



In [ ]:
full = pd.concat([train_df,test_df])

In [ ]:
full.Name = full.Name.fillna('NoRecord')

In [ ]:
full['last_name'] =[a[1] if len(a)>1 else 'NoRecord' for a in full.Name.str.split(' ')]

In [ ]:
full = pd.merge(full,full.groupby(['GroupId','last_name']).group_num.count() , on=['GroupId','last_name'])
full =full.rename(columns={'group_num_x':'group_num','group_num_y':'same_name'})

In [ ]:
full['IsFamily'] = [1 if a>1 and a==b else 0 for a,b in zip(full.group_num,full.same_name)]

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
full['GroupId'] = encoder.fit_transform(full['GroupId'])

In [ ]:
#separate fulldata back to train_df and test_df
train_df = full.query('isTrain==True')
test_df = full.query('isTrain==False')

In [ ]:
#Transported
train_df.Transported.unique()

# Explanatory Data Analysis
<font size=4> Data was preprocessed. Let's do explanatory data analysis. </font>

# HomePlanet

In [ ]:
import seaborn as sns
sns.countplot(full.HomePlanet);

In [ ]:
sns.countplot(x='HomePlanet',hue='Transported',data=train_df);

* Almost half of passengers were from earth
* Somehow more Earth people did not get transported compared with people from Europa and Mars. 

# CryoSleep

In [ ]:
sns.countplot(full.CryoSleep)

In [ ]:
sns.countplot(x='CryoSleep',hue='Transported',data=train_df)

In [ ]:
plt.figure(figsize=(10,4))

ax1 = plt.subplot(1,2,1)
sns.countplot(x='Transported',data=full.query('group_num==1 and CryoSleep==True'));
plt.title('Individual Traveler with CryoSleep');

ax2= plt.subplot(1,2,2)
sns.countplot(x='Transported',data=full.query('group_num>1 and CryoSleep==True'));
plt.title('Group Traveler with CryoSleep');


* <font size=4> Many of CryoSleep passengers were transported.
* <font size=4> The ratiod doesn't change for individual or group traveler. Most likely group traveler choose the same if they take CryoSleep or not.

# Cabin (Deck, Num, Side)

In [ ]:
plt.figure(figsize=(10,4));

ax1 = plt.subplot(1,2,1)
sns.countplot(full.deck);

ax3 = plt.subplot(1,2,2)
sns.countplot(full.side);

In [ ]:
plt.figure(figsize=(10,4))

ax1 = plt.subplot(1,2,1)
sns.countplot(x='deck', hue='Transported',data=full);
plt.title('Transported or Not by deck');

ax2= plt.subplot(1,2,2)
sns.countplot(x='side', hue='Transported',data=full);
plt.title('Transported or Not by Side');


* <font size=4> Not much interesting insight here.

# Destination

In [ ]:
sns.countplot(full.Destination);

# Age

In [ ]:
#binning with range 
bins = [0,3,10,18,25,35,45, 60,80]
train_df["binned_age"] = pd.cut(train_df['Age'], bins)

#binned_age = train_df.binned_age.unique().to_list()

In [ ]:
plt.figure(figsize=(8,8));
ax =sns.countplot(x='binned_age',data=train_df,hue='Transported')
#ax.set_xticklabels(binned_age,rotation=40,ha="right")
plt.title("Age Group Distribution");


In [ ]:
print('Ratio Under 18 with CryoSleep',round(len(full.query('Age<18 and CryoSleep==True'))/len(full.query('Age<18')),3),  '%')
print('Ratio Under 18 with CryoSleep',round(len(full.query('CryoSleep==True'))/len(full),3),  '%')


* <font size=4> Not like Titanic, kids in the spaceship got transported. It might be explained by the ration of CryoSleep. More yourger people used CryoSleep.  

# VIP

In [ ]:
sns.countplot(full.VIP);

In [ ]:
print('Vip Number: ', len(full.query('VIP==True')))
print('Vip with CryoSleep Number: ', len(train_df.query('VIP==True and CryoSleep==True')))

In [ ]:
plt.figure(figsize=(10,4))

ax1 = plt.subplot(1,2,1)
plt.title('Passengers with VIP')
sns.countplot(x='Transported',data=full.query('VIP==True'));

ax2 = plt.subplot(1,2,2)
sns.countplot(x='Transported',data=full.query('VIP==True and CryoSleep==True'));
plt.title('CryoSleep Passengers with VIP')

* <font size=4> Only 273 people use VIP. At first, I didn't get why people with CryoSleep used VIP, but it makes sense after I made a graph. There might be a policy to save them first in emergency.

# Making a Model

In [ ]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split

In [ ]:
# member might have some information
train_df['member'] = train_df['member'].astype(int)
test_df['member'] = test_df['member'].astype(int)


train_df['Transported'] = train_df['Transported'].astype(int)

In [ ]:
train_df.columns

In [ ]:
FEATURES = ['Age',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'GroupId', 'group_num','member',
        'home_embed', 'sleep_embed', 'num',
       'deck_embed', 'side_embed', 'destination_emmbed', 'vip_embed',
        'luxury','luxurywithoutfood','IsFamily','same_name','group_age_mean','group_age_std']
#'nan_count', 'luxury','isShoppingMall','isSpa','isRoomService','isFoodCourt','isAge','isVRDecK','isFamily','same_name'
TARGET = ['Transported']

In [ ]:
X_train,X_val, y_train,y_val = train_test_split(train_df[FEATURES],train_df[TARGET],random_state=1)

In [ ]:
model = XGBClassifier()

history = model.fit(X_train, y_train,eval_set =[(X_train,y_train),(X_val,y_val)],early_stopping_rounds =5,verbose=False)

acc = accuracy_score(y_val.astype(int), model.predict(X_val).astype(int))
print(f"Accuracy:' {round(acc,3)}")


In [ ]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=50,
    random_seed=42,
    learning_rate=0.5,
    custom_loss=['AUC', 'Accuracy']
)

clf.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    verbose=False,
    plot=True
)

acc = accuracy_score(y_val, clf.predict(X_val))
print(f"Accuracy:' {round(acc,3)}")

In [ ]:
results = model.evals_result()
# plot learning curves
plt.figure(figsize=(10, 8))
plt.plot(results['validation_0']['logloss'], label='train')
plt.plot(results['validation_1']['logloss'], label='test')
# show the legend
plt.legend()
plt.xlabel('iterations')
plt.ylabel('mlogloss')
# show the plot
plt.show()

In [ ]:
from xgboost import plot_importance
ax = plot_importance(model,max_num_features=30)
fig = ax.figure
fig.set_size_inches(10,8)
plt.show()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cv = KFold(n_splits=5, random_state=1, shuffle=True)
model = CatBoostClassifier(learning_rate=0.1,grow_policy='SymmetricTree',n_estimators=500,verbose=False)
# evaluate model
scores = cross_val_score(model, train_df[FEATURES], train_df[TARGET], scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print(scores)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cv = KFold(n_splits=5, random_state=1, shuffle=True)
model = XGBClassifier(n_estimators= 300,verbose=False)
# evaluate model
scores = cross_val_score(model, train_df[FEATURES], train_df[TARGET], scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print(scores)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

# Making Submission File

In [ ]:
model = CatBoostClassifier(learning_rate=0.1,n_estimators=400,verbose=False,random_state=1)
model.fit(train_df[FEATURES],train_df[TARGET])

In [ ]:
predictions = model.predict(test_df[FEATURES])

In [ ]:
sub = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
sub['Transported'] = predictions.astype(bool)
sub.to_csv('submission.csv',index=False)

<font size=4> Thank you for reading my notebook!! Hope you learn something new from this. </font>
    
 <font size=6>To be continued...